In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math


from sklearn.tree            import DecisionTreeRegressor
from sklearn.linear_model    import LinearRegression,Ridge,Lasso,RidgeCV, LassoCV, ElasticNetCV
from sklearn.ensemble        import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold,train_test_split 
from sklearn.metrics         import r2_score,mean_squared_error
from sklearn.ensemble        import RandomForestRegressor,  GradientBoostingRegressor 
from xgboost                 import XGBRegressor 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV




In [52]:
train_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\train_processedWithLog1pTarget.csv")

In [53]:
test_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\test_processed.csv")

In [54]:
train_df.isnull().sum()

latitude                             0
longitude                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                     0
log_(price+1)                        0
neighbourhood_group_Bronx            0
neighbourhood_group_Brooklyn         0
neighbourhood_group_Manhattan        0
neighbourhood_group_Queens           0
neighbourhood_group_Staten Island    0
room_type_Entire home/apt            0
room_type_Private room               0
room_type_Shared room                0
dtype: int64

In [55]:
train_df_with_dummies = train_df

In [56]:
X=train_df_with_dummies.drop(['log_(price+1)'],axis=1)
y=train_df_with_dummies['log_(price+1)']

In [57]:
#y= target
#x = nums
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=24)

In [58]:
model = XGBRegressor(random_state=10)
parameters = {'learning_rate': [0.01,0.02],
                  'n_estimators' : [100,200]
                 }


model = XGBRegressor(random_state=10)
parameters = {'learning_rate': [0.01,0.02],
                  'booster' : ['dart'],
                  'sample_type' : ['uniform','weighted'],
                  
                 }


In [59]:
grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 5, n_jobs=2, verbose=9)
grid.fit(X_train, y_train)    

# Results from Grid Search
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")    
    
print("\n The best estimator across ALL searched params:\n",grid.best_estimator_)
    
print("\n The best score across ALL searched params:\n",grid.best_score_)
    
print("\n The best parameters across ALL searched params:\n",grid.best_params_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:   13.8s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:   51.6s
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:  1.3min finished


[11:07:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

 Results from Grid Search 

 The best estimator across ALL searched params:
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=10,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

 The best score across ALL searched params:
 0.5383517234274293

 The best parameters across ALL searched params:
 {'learning_rate': 0.02, 'n_estimators': 200}


In [60]:
xgbmodel = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=10,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [61]:
xgbmodel.fit(X_train,y_train)

[11:08:33] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=10,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [62]:
y_pred = xgbmodel.predict(X_test)
print('Train RMSE:',np.sqrt(mean_squared_error(y_train, xgbmodel.predict(X_train))))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test, xgbmodel.predict(X_test))))

Train RMSE: 0.4682271488797464
Test RMSE: 0.48090056109809554


In [63]:
np.sqrt(mean_squared_error(y_train,np.exp(xgbmodel.predict(X_train))-1)), np.sqrt(mean_squared_error(y_test,np.exp(xgbmodel.predict(X_test))-1))

(123.74870540386064, 124.09448926719773)

In [64]:
y_pred = grid.predict(X_test)
print('Train RMSE:',np.sqrt(mean_squared_error(y_train, grid.predict(X_train))))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test, grid.predict(X_test))))

Train RMSE: 0.4682271488797464
Test RMSE: 0.48090056109809554


In [65]:
np.sqrt(mean_squared_error(y_train,np.exp(grid.predict(X_train))-1)), np.sqrt(mean_squared_error(y_test,np.exp(grid.predict(X_test))-1))

(123.74870540386064, 124.09448926719773)

## TEST DATA

In [66]:
test_df

,id,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,1525602,40.66751,-73.95867,2,121,1.70,1,39,0,1,0,0,0,1,0,0
1,30430185,40.75655,-73.99690,1,18,2.49,30,364,0,0,1,0,0,0,1,0
2,21354525,40.69252,-73.99121,1,87,4.29,1,108,0,1,0,0,0,0,1,0
3,35995074,40.77292,-73.90101,9,0,0.01,2,365,0,0,0,1,0,0,1,0
4,34392081,40.71863,-73.94980,1,28,14.00,2,20,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14664,27730191,40.59446,-73.94289,2,1,0.59,1,36,0,1,0,0,0,1,0,0
14665,29676417,40.74319,-73.98074,2,1,0.16,1,365,0,0,1,0,0,0,0,1
14666,1813829,40.82519,-73.94530,1,93,1.34,3,358,0,0,1,0,0,0,1,0
14667,25017403,40.72238,-73.99466,1,15,1.12,8,34,0,0,1,0,0,1,0,0


In [67]:
test_id_col = test_df['id']

In [68]:
test_df.isnull().sum()

id                                   0
latitude                             0
longitude                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                     0
neighbourhood_group_Bronx            0
neighbourhood_group_Brooklyn         0
neighbourhood_group_Manhattan        0
neighbourhood_group_Queens           0
neighbourhood_group_Staten Island    0
room_type_Entire home/apt            0
room_type_Private room               0
room_type_Shared room                0
dtype: int64

In [69]:
test_df_with_dummies = test_df

In [70]:

test_df_with_dummies = test_df_with_dummies.drop(['id'],axis=1)

In [71]:
test_df_with_dummies.columns

Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'neighbourhood_group_Bronx',
       'neighbourhood_group_Brooklyn', 'neighbourhood_group_Manhattan',
       'neighbourhood_group_Queens', 'neighbourhood_group_Staten Island',
       'room_type_Entire home/apt', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')

In [72]:
test_df_with_dummies['price']= grid.predict(test_df_with_dummies)

In [73]:
test_df_with_dummies['price'] = np.exp(test_df_with_dummies['price'])-1

In [74]:
submissions_xgbmodel = pd.concat([test_id_col, test_df_with_dummies['price']], axis=1) 


In [75]:
submissions_xgbmodel.to_csv('submission_xgbmodel_gridSearchCV_tune.csv', index=False) 

In [76]:
submissions_xgbmodel

,id,price
0,1525602,132.806442
1,30430185,142.094437
2,21354525,77.771812
3,35995074,60.934223
4,34392081,71.945732
...,...,...
14664,27730191,120.272942
14665,29676417,128.613785
14666,1813829,73.036926
14667,25017403,187.152039


In [77]:
submissions_xgbmodel.isnull().sum()

id       0
price    0
dtype: int64